## Importar los Datos de Stackoverflow

### Descarga de los CSV

Descargamos los CSV:

!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.001 -O - > es.stackoverflow.csv.7z
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.002 -O - >> es.stackoverflow.csv.7z

Descomprimimos y borramos el archivo comprimido

!7zr x -aoa es.stackoverflow.csv.7z
!rm es.stackoverflow.csv.7z

### Importar la Base de Datos a PyArrow

In [ ]:
# Probamos la importación automática de Arrow
# Utilizamos datasets para que sea más eficiente en memoria
from pyarrow import csv
import pyarrow.dataset as ds

posts = ds.dataset("Posts.csv", format="csv")
users = ds.dataset("Users.csv", format="csv")
tags = ds.dataset("Tags.csv", format="csv")
votes = ds.dataset("Votes.csv", format="csv")
comments = ds.dataset("Comments.csv", format="csv")

print(f""" - Rows:
Posts: {posts.count_rows()}
Users: {users.count_rows()}
Tags: {tags.count_rows()}
Votes: {votes.count_rows()}
Comments: {comments.count_rows()} """)

 - Rows:
Posts: 410346
Users: 319535
Tags: 3048
Votes: 787607
Comments: 689730 


## Ejercicios
### RQ1: Distribution of Questioners
We plot the histogram of questioners in Figure 3. The graph shows the number of developers that ask a given number of questions, and its y-axis is in log-scale. From the graph, we notice that most developers (33,907 out of 44,087 developers) only ask one question. Only about 23.1% of the developers ask two or more questions. The number of developers that ask questions reduces exponentially as we consider a higher number of posted questions. Only 1.6% of the developers ask more than 5 questions. 

The result shows that there are few “regular” questioners on StackOverflow. This is possibly because many questions have already been asked before and users could find answers to them by just looking into the various pages on StackOverflow or other question and answer sites via search engines

### RQ2: Distribution of Answerers
We plot the histogram of answerers in Figure 3. The graph shows the number of developers that answer a given num- ber of questions and its y-axis is also in log-scale. From the graph, we notice that most developers (28,578 out of 44,087 developers) only answer one question. About one thousand developers (2.3%) do not answer any questions. Only about 35.2% of the developers answer two or more questions. The number of developers that answer questions reduces expo- nentially as we consider a higher number of answers. On- ly 7.8% of the developers answer more than 5 questions. The highest number of questions a developer answers in our dataset is 178. There is only one developer that answers this many questions.

Compared with the distribution of questioners, the distribution of answers is different. The number of developers that answer a substantial number of questions (> 5) is more than the number of developers that ask a substantial number of questions (> 5)—3,424 (7.8%) versus 701 (1.6%). This may imply that many developers on StackOverflow are interested

### RQ3: Segregation of StackOverflow Com munity
To answer this research question, we investigate the proportion of posts that various developers make that are answers to some questions. We show this in Figure 5. We notice that a majority of developers only ask questions but do not answers them (83.2%, 36,672 developers). Thus, we could divide the StackOverflow community into two groups: people that only ask questions, and those that answer one or more questions. The first group is the majority.

We also note another peak in Figure 5: These are developers (2,956 of 44,087 developers), with 50-59% of posts being answers. These correspond to ideal developers that contribute answers to the community as much as requesting answers from the community.

### RQ4: RQ4: Reciprocity in StackOverflow
To answer the fourth research question, we investigate the help graph. A help graph is a directed graph, where each developer is a node, and the node corresponding to a developer D1 is linked to that of D2 if D1 answers a question posted by D2. We would like to investigate how often two developers D1 and D2 are connected by two edges, one from D1 to D2 and the other from D2 to D1. We find that there are only a few of such developers (23 pairs). We highlight a few in Table 1. The table contains the identifiers of the helpers and helpees that reciprocate. From the result, we hypothesize that developers tend to help anyone no matter if they have helped him or her before and StackOverflow tends to benefit the community as a whole.